In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Question 1: Top 10 Bundesliga teams over entire period

**Metric: Points Per Game (PPG)**
- 3 points for win
- 1 point for draw
- 0 points for loss

Same point system as used in real life, ensures fairness, eliminates goal outliers and also gives points for draws. 

In [ ]:
# Load data
df = pd.read_csv('data/bundesliga.csv')

# Convert Date to datetime 
df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True)

# Quick look
print(f"Dataset shape: {df.shape}")
print(f"\nDate range: {df['Date'].min()} to {df['Date'].max()}")
df.head()

In [ ]:
# Check for missing values
df.info()

## Calculate Points for Each Team

In [ ]:
def calculate_team_stats(df):
    stats = {}
    
    # Process matches
    for _, row in df.iterrows():
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        result = row['FTR']
        
        # Initialize teams if not seen before
        if home_team not in stats:
            stats[home_team] = {'games': 0, 'points': 0}
        if away_team not in stats:
            stats[away_team] = {'games': 0, 'points': 0}
        
        # Update home team stats
        stats[home_team]['games'] += 1
        
        if result == 'H': 
            stats[home_team]['points'] += 3
        elif result == 'D':     
            stats[home_team]['points'] += 1
        
        # Update away team stats
        stats[away_team]['games'] += 1
        
        if result == 'A': 
            stats[away_team]['points'] += 3
        elif result == 'D':     
            stats[away_team]['points'] += 1
    
    return stats

In [ ]:
# Calculate stats
team_stats = calculate_team_stats(df)

# Convert to df
stats_df = pd.DataFrame.from_dict(team_stats, orient='index')
stats_df.index.name = 'Team'
stats_df.reset_index(inplace=True)

# Calculate derived metrics
stats_df['PPG'] = stats_df['points'] / stats_df['games']

print(f"Total teams in dataset: {len(stats_df)}")
stats_df.head()

In [ ]:
# Filter teams with min 100 games and get top 10
min_games = 100

filtered_stats = stats_df[stats_df['games'] >= min_games]
top_10 = filtered_stats.sort_values('PPG', ascending=False).head(10)

print(f"=== TOP 10 BUNDESLIGA TEAMS (1993-2018) ===")
print(f"Filtered by minimum {min_games} games | Teams remaining: {len(filtered_stats)}\n")

top_10[['Team', 'games', 'PPG']]

In [ ]:
# Bar chart of top 10 teams
fig, ax = plt.subplots(figsize=(12, 8))

# PPG comparison
ax.barh(range(len(top_10)), top_10['PPG'].values, color='steelblue')
ax.set_yticks(range(len(top_10)))
ax.set_yticklabels(top_10['Team'].values)
ax.set_xlabel('Points Per Game', fontsize=12)
ax.set_title('Top 10 Bundesliga Teams by PPG (1993-2018)', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

# Add value labels
for i, v in enumerate(top_10['PPG'].values):
    ax.text(v + 0.02, i, f'{v:.3f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

## Question 2: Total goals prediction

A model is implemented to predict total goals in football matches. Since bets are placed at halftime the target variable for prediction is the total goals in the second half of the matches. 

In [ ]:
# Drop rows with missing halftime data (1993/94 and 1994/95 seasons)
df_clean = df.dropna(subset=['HTHG', 'HTAG', 'HTR']).copy()

print(f"\nAfter dropping missing halftime data: {df_clean.shape}")
print(f"Rows dropped: {len(df) - len(df_clean)}")

In [ ]:
# Target variable: Total goals in second half
df_clean['second_half_goals'] = (df_clean['FTHG'] - df_clean['HTHG']) + (df_clean['FTAG'] - df_clean['HTAG'])

# Halftime features
df_clean['ht_total_goals'] = df_clean['HTHG'] + df_clean['HTAG']
df_clean['ht_goal_diff'] = df_clean['HTHG'] - df_clean['HTAG']

df_clean['year'] = df_clean['Date'].dt.year

# Calculate running average of second half goals for each team.
def calculate_historical_avg(df, team_col):
    is_home = team_col == 'HomeTeam'
    team_avgs = {}
    result = []
    
    for _, row in df.iterrows():
        team = row[team_col]
        
        # Get average from prior matches only
        if team in team_avgs and team_avgs[team]['games'] > 0:
            avg = team_avgs[team]['sh_cumulative_goals'] / team_avgs[team]['games']
        else:
            avg = np.nan
        
        result.append(avg)
        
        # Update running totals after recording the average
        if team not in team_avgs:
            team_avgs[team] = {'sh_cumulative_goals': 0, 'games': 0}
        
        sh_goals = row['FTHG'] - row['HTHG'] if is_home else row['FTAG'] - row['HTAG']
        team_avgs[team]['sh_cumulative_goals'] += sh_goals
        team_avgs[team]['games'] += 1
    
    return result

# Calculate historical averages
df_clean['home_team_hist_sh_goals'] = calculate_historical_avg(df_clean, 'HomeTeam')
df_clean['away_team_hist_sh_goals'] = calculate_historical_avg(df_clean, 'AwayTeam')

# Fill NaNs: avg. about 0.8 goals per team per second half
AVG_SH_GOALS_PER_TEAM = 0.8

df_clean['home_team_hist_sh_goals'] = df_clean['home_team_hist_sh_goals'].fillna(AVG_SH_GOALS_PER_TEAM)
df_clean['away_team_hist_sh_goals'] = df_clean['away_team_hist_sh_goals'].fillna(AVG_SH_GOALS_PER_TEAM)

df_clean['combined_hist_sh_goals'] = df_clean['home_team_hist_sh_goals'] + df_clean['away_team_hist_sh_goals']

In [ ]:
# Split: Train on data before 2016, test on 2016-2018
train_cutoff_year = 2016
train_df = df_clean[df_clean['year'] < train_cutoff_year].copy()
test_df = df_clean[df_clean['year'] >= train_cutoff_year].copy()

print(f"\n=== Train-Test Split ===")
print(f"Train set: {len(train_df)} matches ({train_df['year'].min()}-{train_df['year'].max()})")
print(f"Test set: {len(test_df)} matches ({test_df['year'].min()}-{test_df['year'].max()})")

# Features
feature_cols = [
    'ht_total_goals', 'ht_goal_diff',  # Half time features
    'home_team_hist_sh_goals',         # Historical: home team's avg 2nd half goals
    'away_team_hist_sh_goals',         # Historical: away team's avg 2nd half goals
]

X_train = train_df[feature_cols]
y_train = train_df['second_half_goals']

X_test = test_df[feature_cols]
y_test = test_df['second_half_goals']

print(f"\nFeatures used: {len(feature_cols)}")
for f in feature_cols:
    print(f"  - {f}")
X_train.describe()
X_test.describe()



In [ ]:
# Baseline model: Predict mean of training 
baseline_pred = np.full(len(y_test), y_train.mean())
baseline_mae = mean_absolute_error(y_test, baseline_pred)
baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_pred))

print(f"\n=== Baseline (Predict Mean) ===")
print(f"Mean second half goals (train): {y_train.mean():.3f}")
print(f"MAE: {baseline_mae:.3f}")
print(f"RMSE: {baseline_rmse:.3f}")

In [ ]:
# Random Forest Regressor
print(f"\nTraining Random Forest")
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42,
)
rf_model.fit(X_train, y_train)

rf_pred_train = rf_model.predict(X_train)
rf_pred_test = rf_model.predict(X_test)

rf_mae_train = mean_absolute_error(y_train, rf_pred_train)
rf_rmse_train = np.sqrt(mean_squared_error(y_train, rf_pred_train))
rf_r2_train = r2_score(y_train, rf_pred_train)

rf_mae_test = mean_absolute_error(y_test, rf_pred_test)
rf_rmse_test = np.sqrt(mean_squared_error(y_test, rf_pred_test))
rf_r2_test = r2_score(y_test, rf_pred_test)

print(f"\nTrain Performance:")
print(f"  MAE: {rf_mae_train:.3f}")
print(f"  RMSE: {rf_rmse_train:.3f}")
print(f"  R²: {rf_r2_train:.3f}")

print(f"\nTest Performance:")
print(f"  MAE: {rf_mae_test:.3f}")
print(f"  RMSE: {rf_rmse_test:.3f}")
print(f"  R²: {rf_r2_test:.3f}")

In [ ]:
# Residual Plot: Test Set
residuals = y_test - rf_pred_test

fig, ax = plt.subplots(figsize=(8, 5))

scatter = ax.scatter(rf_pred_test, residuals, c=y_test, cmap='viridis', alpha=0.6)
ax.axhline(0, color='red', linestyle='--')
ax.set_xlabel('Predicted Second Half Goals')
ax.set_ylabel('Residuals (Actual - Predicted)')
ax.set_title('Residual Plot (colored by actual goals)')
plt.colorbar(scatter, ax=ax, label='Actual Goals')

plt.tight_layout()
plt.show()

In [ ]:
#Feature importance plot
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n=== Feature Importance (Random Forest) ===")
print(feature_importance.to_string(index=False))

plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_importance)), feature_importance['importance'].values)
plt.yticks(range(len(feature_importance)), feature_importance['feature'].values)
plt.xlabel('Importance')
plt.title('Random Forest Feature Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()